In [2]:
# -*- coding: utf-8 -*-
"""
#Script for data cleanup of US Accident data

Created on Mon Apr 20 21:11:18 2020
@author: keino
"""
### Description of US Countrywide Traffic Accident Dataset(2016 - 2019)
# Source: https://www.kaggle.com/sobhanmoosavi/us-accidents/download

# Import Required Libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import scipy.stats as stats
import datetime
import nltk
from IPython.display import display
import statsmodels.api as sm
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

## Load the dataset
#sys.path.append('/content/drive/My Drive/MSSM/ML_for_BDS')   #change working directory
db = pd.read_csv('/Users/ShaunPorwal/Desktop/sinai_classes/BMI3002_ML_2020/MLProject/US_Accidents_Dec19.csv')

In [3]:
### Step #1. Fix Date and Time Columns
#Since there is time and date columns, we can make it easier to work with (taken from Ismael)
# Convert Start_Time and End_Time to datetypes
db['Start_Time'] = pd.to_datetime(db['Start_Time'], errors='coerce')
db['End_Time'] = pd.to_datetime(db['End_Time'], errors='coerce')

In [4]:
# Extract year, month, day, hour and weekday
db['Year']=db['Start_Time'].dt.year
db['Month']=db['Start_Time'].dt.strftime('%b')
db['Day']=db['Start_Time'].dt.day
db['Hour']=db['Start_Time'].dt.hour
db['Weekday']=db['Start_Time'].dt.strftime('%a')
db.head(2)

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Weekday
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,Night,Night,Night,Night,2016,Feb,8,5,Mon
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,Night,Night,Night,Day,2016,Feb,8,6,Mon


In [5]:
#Check missing remaining
print(db.isnull().sum()>0)

ID                       False
Source                   False
TMC                       True
Severity                 False
Start_Time               False
End_Time                 False
Start_Lat                False
Start_Lng                False
End_Lat                   True
End_Lng                   True
Distance(mi)             False
Description               True
Number                    True
Street                   False
Side                     False
City                      True
County                   False
State                    False
Zipcode                   True
Country                  False
Timezone                  True
Airport_Code              True
Weather_Timestamp         True
Temperature(F)            True
Wind_Chill(F)             True
Humidity(%)               True
Pressure(in)              True
Visibility(mi)            True
Wind_Direction            True
Wind_Speed(mph)           True
Precipitation(in)         True
Weather_Condition         True
Amenity 

In [6]:
# Extract the amount of time in the unit of minutes for each accident, 
# round to the nearest integer
td='Time_Duration(min)'
db[td]=round((db['End_Time']-db['Start_Time'])/np.timedelta64(1,'m'))
#db.info()

In [7]:
# Drop the rows with td<0
neg_outliers=db[td]<=0

# Drop rows with negative td
db.dropna(subset=[td],axis=0,inplace=True)

# Double check to make sure no more negative td
db[td].loc[(db[td]>=0)]

#Check missing remaining
#print(db.isnull().sum())

0          314.0
1           30.0
2           30.0
3           30.0
4           30.0
           ...  
2974330     29.0
2974331     27.0
2974332     28.0
2974333     29.0
2974334     29.0
Name: Time_Duration(min), Length: 2974318, dtype: float64

In [8]:
### Step #2. Deal with tricky geography columns
#Drop End Lat and End Lng, since too many are missing and are hard to impute.
#They add little value
db.drop(labels=['End_Lat', 'End_Lng'],axis=1,inplace=True)

#Fill missing street number with a zero
#db['Number'] = db['Number'].fillna(0)
db.drop(labels=['Number'],axis=1,inplace=True)

#Impute the timezone, Zipcode and Airport_Code based on the State column
db['Timezone'] = db.groupby('State')['Timezone'].transform(lambda tz: tz.fillna(tz.value_counts().index[0]))
db['Zipcode'] = db.groupby('State')['Zipcode'].transform(lambda zc: zc.fillna(zc.value_counts().index[0]))
db['Airport_Code'] = db.groupby('State')['Airport_Code'].transform(lambda ac: ac.fillna(ac.value_counts().index[0]))

#Impute missing city uing most occuring states
db['City'] = db.groupby('State')['City'].transform(lambda grp: grp.fillna(grp.value_counts().index[0]))

#Check missing remaining
print(db.isnull().sum())


ID                             0
Source                         0
TMC                       728071
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
Distance(mi)                   0
Description                    1
Street                         0
Side                           0
City                           0
County                         0
State                          0
Zipcode                        0
Country                        0
Timezone                       0
Airport_Code                   0
Weather_Timestamp          36705
Temperature(F)             56063
Wind_Chill(F)            1852623
Humidity(%)                59173
Pressure(in)               48142
Visibility(mi)             65691
Wind_Direction             45101
Wind_Speed(mph)           440840
Precipitation(in)        1998358
Weather_Condition          65932
Amenity                        0
Bump      

In [9]:
### Step #3. Fix all weather related columns
# Fill in NaN with mean values where appropriate
fill = ['Temperature(F)','Pressure(in)', 'Humidity(%)','Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)', 'Visibility(mi)']
for f in fill:
    db[f]=db[f].fillna(db[f].mean())


In [10]:
### Step #4. Fix all other columns   
#Impute time of day-sih columns 
def median_imputer(x):
    db[x].fillna(db[x].mode()[0],inplace=True)

median_impute = ['Sunrise_Sunset','Civil_Twilight','Astronomical_Twilight','Wind_Direction','Weather_Condition']
for col in median_impute:
    median_imputer(col)

In [11]:
# Impute Nautical Twilight, using start time
def fill(db,columns):
    lst = db[db[columns].isna()].index
    for i in lst:
        if 6<= db.loc[i,'Start_Time'].hour and db.loc[i,'Start_Time'].hour <18:
            db[columns] = db[columns].fillna('Day')
        else:
            db[columns] = db[columns].fillna('Night')

fill(db,'Nautical_Twilight')

In [12]:
#weather stamp not really important but can impute by using start time
db.loc[(pd.isnull(db.Weather_Timestamp)), 'Weather_Timestamp'] = db.Start_Time

#Fill one record in Description with the word 'Accident' #can fill later with most occuring word
db.Description = db.Description.fillna('Accident')

#Check missing remaining
print(db.isnull().sum())

ID                            0
Source                        0
TMC                      728071
Severity                      0
Start_Time                    0
End_Time                      0
Start_Lat                     0
Start_Lng                     0
Distance(mi)                  0
Description                   0
Street                        0
Side                          0
City                          0
County                        0
State                         0
Zipcode                       0
Country                       0
Timezone                      0
Airport_Code                  0
Weather_Timestamp             0
Temperature(F)                0
Wind_Chill(F)                 0
Humidity(%)                   0
Pressure(in)                  0
Visibility(mi)                0
Wind_Direction                0
Wind_Speed(mph)               0
Precipitation(in)             0
Weather_Condition             0
Amenity                       0
Bump                          0
Crossing

In [16]:
db['Severity'].value_counts()

2    1468069
3     772423
4       4804
1        968
Name: Severity, dtype: int64

In [14]:
#Give missing TMC codes, the US general code of 201
#db.TMC = db.TMC.fillna(201.0)
#Check missing remaining
#print(db.isnull().sum())

#Option #2: remove columns with missing TMC codes.
TMC = 'TMC'
db.dropna(subset=[TMC],axis=0,inplace=True)

#Check missing remaining
print(db.isnull().sum())

ID                       0
Source                   0
TMC                      0
Severity                 0
Start_Time               0
End_Time                 0
Start_Lat                0
Start_Lng                0
Distance(mi)             0
Description              0
Street                   0
Side                     0
City                     0
County                   0
State                    0
Zipcode                  0
Country                  0
Timezone                 0
Airport_Code             0
Weather_Timestamp        0
Temperature(F)           0
Wind_Chill(F)            0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Precipitation(in)        0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
S

In [15]:
db.shape

(2246264, 52)

In [16]:
#Save df of cleaned data compressed since dataset is large.
compression_opts = dict(method='zip',
                        archive_name='out.csv') 
db.to_csv('/Users/ShaunPorwal/Desktop/sinai_classes/BMI3002_ML_2020/MLProject/clean_US_accidents_2019.zip', compression=compression_opts)

In [ ]:
# Or not lol
#db.to_csv('/content/drive/My Drive/MSSM/ML_BDS/2_clean_US_accidents_2019.csv')